In [1]:
import pandas as pd
df = pd.read_pickle('yonhap_ready.pkl')

In [2]:
df.head(3)

,article,title,sec1,sec_idx
0,'회장이 청부폭력' 루머 협박범도 기소…사이버명예훼손 전담수사 첫 사례 ...,"""해경이 가만있으라 방송"" 허위사실 유포 40대 기소",사회,4
1,지난 해 9월 북한의 함경북도 항구도시 나진과 러시아 극동지역 도시 ...,南 점검단 방북…나진-하산 프로젝트 시범운송 시작(종합),정치,6
2,포항 도착한 화물선 러시아 시베리아산 유연탄 4만 500t을...,나진-하산 프로젝트 탄력…시범수송 석탄 포항 도착(종합),경제,0


In [3]:
X = df.article
y = df.sec_idx
print(X.shape)
print(y.shape)

(442481,)
(442481,)


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(331860,)
(110621,)
(331860,)
(110621,)


In [5]:
%%time
from konlpy.tag import Mecab; m = Mecab()
#pos = lambda d: ['/'.join(p) for p in m.pos(d)]
def noun_tagger(text):
    #return [pos[0] for pos in m.pos(text) if (pos[1] in ['SL','SH','SN']) or pos[1].startswith('NN')]
    return [pos[0] for pos in m.pos(text) if pos[1].startswith('NN')]

CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 43 ms


### Summary:
    * vect.fit(train) learns the vocabulary of the training data
    * vect.transform(train) uses the fitted vocabulary to build a document term matrix from the training data
    * vect.transform(test) uses the fitted vocabulary to build a document term matrix from the testing data(and ignores tokens it hasn't seen before)

In [6]:
stopwords_news=['지난해', '중','말', '뒤', '곳', '군', '위', '개', '간', '건', '이날', '도', '등', '명', '시', '앞', '원', '분', '회', '년', '것', '씨', '일', '월','오전','오후']

In [8]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(tokenizer=noun_tagger,stop_words=stopwords_news, ngram_range=(1,3), min_df=2, max_df=.5)
X_train_dtm = vect.fit_transform(X_train)

CPU times: user 20min 9s, sys: 14.1 s, total: 20min 23s
Wall time: 20min 24s


In [9]:
X_test_dtm = vect.transform(X_test)

In [10]:
X_train_dtm.shape

(331860, 9077088)

In [11]:
X_test_dtm.shape

(110621, 9077088)

## save vector & dtm

https://stackoverflow.com/questions/29788047/keep-tfidf-result-for-predicting-new-content-using-scikit-for-python
https://stackoverflow.com/questions/32764991/how-do-i-store-a-tfidfvectorizer-for-future-use-in-scikit-learn

In [12]:
import pickle

with open("yonhap_tfidf_vect.pkl", 'wb') as handle:
    pickle.dump(vect, handle)

with open("yonhap_dtm.pkl", 'wb') as handle:
    pickle.dump(X_test_dtm, handle)

In [13]:
# import and instantiate a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB(alpha=.01)

In [14]:
# train the model using X_train_dtm (timing it with an IPython "magic command")
%time nb.fit(X_train_dtm, y_train)

CPU times: user 4.75 s, sys: 600 ms, total: 5.35 s
Wall time: 5.35 s


MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [15]:
# make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

In [16]:
# calculate accuracy of class predictions
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.86220518708021077

In [17]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_class, target_names=['경제', '과학', '국제', '문화', '사회', '생활', '정치']))

             precision    recall  f1-score   support

         경제       0.91      0.89      0.90     28695
         과학       0.60      0.57      0.58       625
         국제       0.86      0.89      0.87     22358
         문화       0.72      0.89      0.80      2160
         사회       0.83      0.86      0.85     30081
         생활       0.53      0.75      0.62      2187
         정치       0.92      0.82      0.87     24515

avg / total       0.87      0.86      0.86    110621



In [18]:
import numpy as np
counts = np.bincount(y_pred_class[(y_pred_class!=0) & (y_test==0)])
print(np.argmax(counts))

4


In [19]:
X_test[(y_pred_class==5) & (y_test==0)]

239183             보험硏 이정택 위원 분석…"나이 많으면 저체중아 낳을 가능성 커져"...
2993             파키스탄 카라치에서 더위를 식히는 아이들   "온난화로 세계경제 23%...
318317          농촌진흥청 연구 결과…수입산보다 기능성물질 다량 함유   국내산 콩 배아...
405366          강원대 장애라 교수팀 연구 결과   한우고기에 대장암 세포의 증식을 억제...
196634               '보건 안보'에 심각한 위협이 될 수 있는 항생제 오남용과 이에...
356070           감귤의 기능성분이 피부 탄력 증진에 효과가 있다는 연구결과가 나왔다. ...
10583                   뚱뚱한 남자들이 마른 남성들보다 돈을 더 많이 벌고 여자는...
330193          정민유 한국식품연구원 박사팀 실험결과     도토리가 비만 예방에 탁월한...
363809           벌꿀의 한 종류인 '아카시아꿀'이 헬리코박터균을 억제하는 것으로 확인됐...
247903             "발효 고추장 투여 동물실험서 체중·지방 감소 입증"       한...
130419               신생아의 소두증(小頭症)을 유발하는 것으로 의심되는 지카 바이러...
435332          농진청·대학 3곳 공동연구 결과   닭의 몸속에 침투한 조류인플루엔자(A...
168879             듀오 '재혼통계 보고서'…10명중 9명 30∼40대, 평균 9.4개...
12242            원주서 '치악산 배 축제' <>        맛과 품질을 자랑하는 치악...
366890          식품연구원 김효진 박사 연구팀   만성 신장질환에 효능이 있는 프로바이오...
418132           농촌진흥청은 고온에 잘 적응하는 오리 유전자를 발견했다고 20일 밝혔다...
10460                   임피리얼 팰리스 서울의 이탈리

In [22]:
df[df.index==116500]

,level_0,contents,datetime,title,url,label_num,sentences,sen_length,con_length
116500,OB,삼성 라이온즈가 운명의 8회에 또다시 역전극을 만들어내며 4연승을 내달렸다.삼성은 ...,2015-08-19 21:57:03,"삼성, 두산에 6대4 승리, 4경기 연속 역전승",http://sports.news.naver.com/kbaseball/news/re...,0,삼성 라이온즈가 운명의 8회에 또다시 역전극을 만들어내며 4연승을 내달렸다.삼성은 ...,861,1059


In practice Multinomial NB can be better than Linear SVM in some situations and svm better than NB on other datasets. Apparently t's possible to combine the two approaches to get a very good baseline: projects:nbsvm - Sida I. Wang

You can also try non-linear SVMs but often the quadratic complexity of the SMO algorithm (for instance as implemented in libsvm) makes it not practical on datasets with more than 5000 documents. Instead on prefer to use liblinear than can only train linear SVM on large datasets.

https://www.quora.com/What-are-the-differences-similarities-between-SVM-Naive-Bayes-for-binary-text-classification-wrt-how-they-are-processing-the-features

http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC

In [20]:
from sklearn import svm
#clf = svm.SVC(decision_function_shape='ovo')
clfrSVM=svm.LinearSVC(C=1)

In [21]:
%time clfrSVM.fit(X_train_dtm, y_train)

CPU times: user 1min 55s, sys: 764 ms, total: 1min 56s
Wall time: 1min 56s


LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [22]:
y_pred_class_svm = clfrSVM.predict(X_test_dtm)

In [23]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class_svm)

0.90010034261125826

In [24]:
print(classification_report(y_test, y_pred_class_svm, target_names=['경제', '과학', '국제', '문화', '사회', '생활', '정치']))

             precision    recall  f1-score   support

         경제       0.90      0.94      0.92     28695
         과학       0.71      0.59      0.64       625
         국제       0.89      0.91      0.90     22358
         문화       0.86      0.83      0.84      2160
         사회       0.91      0.91      0.91     30081
         생활       0.79      0.60      0.68      2187
         정치       0.92      0.87      0.90     24515

avg / total       0.90      0.90      0.90    110621



https://stackoverflow.com/questions/40115043/no-space-left-on-device-error-while-fitting-sklearn-model

In [25]:
with open("yonhap_svm_clf.pkl", 'wb') as handle:
    pickle.dump(clfrSVM, handle)

In [26]:
with open("yonhap_nb_clf.pkl", 'wb') as handle:
    pickle.dump(nb, handle)